In [1]:
!pip install elasticsearch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.9/431.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.9 MB/s eta 0:00:00


elastic deployment

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    cloud_id="My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGIzZTIzNTFmMTRiNDRhY2E5ZGY1NTY4OTE3YTY5YzE0JDliN2U2M2VhZWM5NjQwOTFhN2U1MDVjMjQ2ZDc4NzNk",
    api_key="NGRfN1k0MEJBa3h0bzRIQWFYblQ6dHlCNzkwX2NUNEd0OXNmeXRsamJkQQ"
)


In [3]:
es.ping()

True

In [4]:
def emptyIndex(indexName):
    query = {
        "query": {
            "match_all": {}
        }
    }
    es.delete_by_query(index=indexName, body=query)


# **All Data**

In [5]:
import json

with open('final_scraped_data(1).json') as f:
    data = json.load(f)

In [6]:
emptyIndex('total_index')

In [7]:
for product in data:
    es.index(index='total_index', body=product["Product Data"])

In [8]:
# Refresh the index to make the data available for search immediately (optional)
es.indices.refresh(index='total_index')

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

In [9]:
# Check if the data is successfully indexed
response = es.search(index='total_index', body={"query": {"match_all": {}}})
print(response)

{'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 100, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'total_index', '_id': 'Queqc40B-Q_i0rt4QEzE', '_score': 1.0, '_ignored': ['Product Reviews.3.keyword', 'Product Reviews.13.keyword', 'Related Products.Product Reviews.5.keyword', 'Product Reviews.11.keyword', 'Related Products.Product Reviews.7.keyword', 'Related Products.Product Reviews.10.keyword', 'Product Description.keyword', 'Related Products.Product Reviews.2.keyword', 'Product Reviews.7.keyword', 'Product Reviews.1.keyword', 'Product Reviews.6.keyword', 'Related Products.Product Description.keyword', 'Related Products.Product Reviews.4.keyword', 'Product Reviews.5.keyword', 'Product Reviews.12.keyword', 'Related Products.Product Reviews.3.keyword'], '_source': {'Product Name': 'PlayStation 5 Console -  Marvel’s Spider-Man 2 Bundle (slim)', 'Product Price': '499', 'Product Rating': '4.5', 'P

In [10]:
response = es.search(index='total_index', body={"query": {"match_all": {}}}, size=5)
for hit in response['hits']['hits']:
    print(hit['_id'], hit['_source']['Product Name'])

Queqc40B-Q_i0rt4QEzE PlayStation 5 Console -  Marvel’s Spider-Man 2 Bundle (slim)
Q-eqc40B-Q_i0rt4QUxB Meta Quest 2 — Advanced All-In-One Virtual Reality Headset — 128 GB
ROeqc40B-Q_i0rt4QUx1 Xbox Series X 1TB SSD Console - Includes Wireless Controller - Up to 120 frames per second - 16GB RAM 1TB SSD - Experience True 4K Gaming Velocity Architecture
Reeqc40B-Q_i0rt4QUzt Meta Quest 3 128GB— Breakthrough Mixed Reality — Powerful Performance — Asgard’s Wrath 2 Bundle
Rueqc40B-Q_i0rt4Qkwh Nintendo Switch™ with Neon Blue and Neon Red Joy‑Con™


<ipython-input-10-1cd0785e0fd7>:1: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific paremeters.
  response = es.search(index='total_index', body={"query": {"match_all": {}}}, size=5)


In [12]:
from elasticsearch import Elasticsearch

def find_similar_products(es_id, num_results):

    # Use More Like This Query to find similar products
    similar_products_query = {
        "size": num_results,  # Adjust the number of results
        "query": {
            "more_like_this": {
                "fields": ["Product Name", "Product Description^3"],  # Boosting relevant fields
                "like": [
                    {
                        "_index": "total_index",
                        "_id": es_id
                    }
                ],
                "min_term_freq": 1,
                "max_query_terms": 25,
                "min_doc_freq": 2,
                "max_doc_freq": 100,
                "minimum_should_match": "30%",
                "boost_terms": 1.5
            }
        }
    }

    # Execute the query
    response = es.search(index='total_index', body=similar_products_query)
    similar_products = [hit['_source'] for hit in response['hits']['hits']]

    # Combine with related products data (optional)
    input_product = es.get(index='total_index', id=es_id)
    related_products = input_product['_source'].get('Related Products', [])
    combined_recommendations = {p['Product Name']: p for p in similar_products + related_products}

    return list(combined_recommendations.values())[:num_results]

# Example usage
es_id = 'Queqc40B-Q_i0rt4QEzE'
num_results = 10  # Number of similar products to retrieve
recommendations = find_similar_products(es_id, num_results)
for rec in recommendations:
    print(rec['Product Name'], rec.get('Product Price'))

MARVEL’S SPIDER-MAN 2 – PS5 Launch Edition 69
Sony Playstation PS4 1TB Black Console 344
Microsoft Bluetooth Elite Series 2 Controller - Starter Bundle for Xbox One 215
Xbox Series S – Starter Bundle - Includes hundreds of games with Game Pass Ultimate 3 Month Membership - 512GB SSD All-Digital Gaming Console None
Final Fantasy XVI - PlayStation 5 39
Meta Quest 3 128GB— Breakthrough Mixed Reality — Powerful Performance — Asgard’s Wrath 2 Bundle 499
God of War Ragnarök - PlayStation 5 39
EA SPORTS FC 24 - PlayStation 5 34
AceGamer Wireless Controller for PS4 with 2 Thumb Grips, 3.5mm Audio and Turbo Function, Purple Galaxy Custom Design V2 Gamepad Joystick for PS4, Compatible with PS4, Slim, Pro and Windows PC 26
NBA 2K24 Kobe Bryant Edition - PlayStation 5 33


In [13]:
import random

# Retrieve products from Elasticsearch
response = es.search(index='total_index', body={"query": {"match_all": {}}}, size=20)
products = [(hit['_id'], hit['_source']['Product Name']) for hit in response['hits']['hits']]

# Select 5 random products
random_products = random.sample(products, 5)

# Find and print similar products for each of the random products
for product_id, product_name in random_products:
    print(f"Random Product: '{product_name}'")
    similar_products = find_similar_products(product_id, 20)  # This function needs to return only product names
    similar_product_names = [product["Product Name"] for product in similar_products]  # Extracting product names
    print(f"Similar products: {similar_product_names}")

<ipython-input-13-feebd9e33ad0>:4: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific paremeters.
  response = es.search(index='total_index', body={"query": {"match_all": {}}}, size=20)


Random Product: 'Just Dance 2024 Edition - Amazon Exclusive Bundle | Nintendo Switch (Code in Box & Ubisoft Connect Code)'
Similar products: ['Nintendo Switch - Animal Crossing: New Horizons Edition - Switch', 'Nintendo Switch – OLED Model - The Legend of Zelda: Tears of the Kingdom Edition', 'Nintendo Switch™ - OLED Model: Super Smash Bros.™ Ultimate Bundle (Full Game Download + 3 Mo. Nintendo Switch Online Membership Included)', 'Meta Quest 2 — Advanced All-In-One Virtual Reality Headset — 128 GB', 'Meta Quest 2 — Advanced All-In-One Virtual Reality Headset — 256 GB', 'PlayStation 5 Console -  Marvel’s Spider-Man 2 Bundle (slim)', 'MARVEL’S SPIDER-MAN 2 – PS5 Launch Edition', 'Mario + Rabbids Sparks of Hope – Standard Edition', 'NBA 2K24 Kobe Bryant Edition - PlayStation 5', 'Microsoft Bluetooth Elite Series 2 Controller - Starter Bundle for Xbox One', 'Meta Quest 3 128GB— Breakthrough Mixed Reality — Powerful Performance — Asgard’s Wrath 2 Bundle', 'Elgato Stream Deck MK.2 – Studio 

**To save the random files in a json for getting our ground truth**






In [ ]:
file_name = 'data.json'

# Open the file for writing
with open(file_name, 'w') as file:
    # Write the data to the file in JSON format
    json.dump(random_items, file, indent=4)

In [15]:
import random
random_items = random.sample(data, 20)
for randitem in random_items:
    print(randitem["Product Index"])

40
56
7
57
33
53
75
81
93
68
26
69
88
12
67
1
96
91
35
42


In [16]:
import json

with open('rand_data.json') as f:
    random_items = json.load(f)

In [17]:
emptyIndex('rand_index')

In [18]:
# Index each product in Elasticsearch
for product in random_items:
    es.index(index='rand_index', body=product["Product Data"])

In [19]:
# Refresh the index to make the data available for search immediately (optional)
es.indices.refresh(index='rand_index')

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

In [20]:
# Check if the data is successfully indexed
response = es.search(index='rand_index', body={"query": {"match_all": {}}})
print(response)

{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 20, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'rand_index', '_id': 'yufBc40B-Q_i0rt4NEy_', '_score': 1.0, '_ignored': ['Product Reviews.8.keyword', 'Product Description.keyword', 'Product Reviews.3.keyword', 'Product Reviews.7.keyword', 'Product Reviews.1.keyword', 'Product Reviews.4.keyword', 'Product Reviews.6.keyword', 'Product Reviews.2.keyword', 'Product Reviews.5.keyword'], '_source': {'Product Name': 'Super Mario Odyssey - US Version', 'Product Price': None, 'Product Rating': '4.8', 'Product Number of Rates': '32,906 ratings', 'Product Image Address': 'https://m.media-amazon.com/images/I/91SF0Tzmv4L._SY445_.jpg', 'Product Description': " Explore 3D kingdoms filled with secrets and surprises, including costumes for Mario and several ways to interact with environments Mario's new friend, Cappy, lets you master new moves like cap throw, cap

In [21]:
count = es.count(index="rand_index")['count']
print("Total documents in index:", count)


Total documents in index: 20


In [65]:
response = es.search(index='rand_index', body={"query": {"match_all": {}}}, size=20)
for i, hit in enumerate(response['hits']['hits']):
    print(i)
    print(hit['_id'], hit['_source']['Product Name'])


0
JOcJc40B-Q_i0rt4lUv1 Super Mario Odyssey - US Version
1
JecJc40B-Q_i0rt4lksh Logitech G Pro Wireless Gaming Mouse with Esports Grade Performance, Black
2
JucJc40B-Q_i0rt4lktN The Legend of Zelda: Breath of the Wild - US Version
3
J-cJc40B-Q_i0rt4lkt9 RoboCop Rogue City (PS5)
4
KOcJc40B-Q_i0rt4lkuh Super Mario 3D World + Bowser's Fury - US Version
5
KecJc40B-Q_i0rt4lkvF Kirby and the Forgotten Land - US Version
6
KucJc40B-Q_i0rt4lkvn Nintendo Neon Purple/ Neon Orange Joy-Con (L-R) - Switch
7
K-cJc40B-Q_i0rt4l0sI Logitech G923 Racing Wheel and Pedals, TRUEFORCE up to 1000 Hz Force Feedback, Responsive Driving Design, Dual Clutch Launch Control, Genuine Leather Wheel Cover, for PS5, PS4, PC, Mac - Black
8
LOcJc40B-Q_i0rt4l0sv Xbox Wireless Headset – Xbox Series X|S, Xbox One, and Windows Devices
9
LecJc40B-Q_i0rt4l0tT Logitech G920 Driving Force Racing Wheel and Floor Pedals, Real Force Feedback, Stainless Steel Paddle Shifters, Leather Steering Wheel Cover for Xbox Series X|S, Xbox One

<ipython-input-65-2cf363ac6628>:1: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific paremeters.
  response = es.search(index='rand_index', body={"query": {"match_all": {}}}, size=20)


In [51]:
from elasticsearch import Elasticsearch

def find_similar_products(es_id, num_results):

    # Use More Like This Query to find similar products
    similar_products_query = {
        "size": num_results,  # Adjust the number of results
        "query": {
            "more_like_this": {
                "fields": ["Product Name", "Product Description^3"],  # Boosting relevant fields
                "like": [
                    {
                        "_index": "rand_index",
                        "_id": es_id
                    }
                ],
                "min_term_freq": 1,
                "max_query_terms": 25,
                "min_doc_freq": 2,
                "max_doc_freq": 100,
                "minimum_should_match": "30%",
                "boost_terms": 1.5
            }
        }
    }

    # Execute the query
    response = es.search(index='rand_index', body=similar_products_query)
    similar_products = [hit['_source'] for hit in response['hits']['hits']]

    # Combine with related products data (optional)
    input_product = es.get(index='rand_index', id=es_id)
    related_products = input_product['_source'].get('Related Products', [])
    combined_recommendations = {p['Product Name']: p for p in similar_products + related_products}

    return list(combined_recommendations.values())[:num_results]

response = es.search(index='rand_index', body={"query": {"match_all": {}}}, size=20)

for hit in response['hits']['hits']:
    print("product:" , hit['_source']['Product Name'])
    es_id = hit['_id']
    num_results = 20  # Number of similar products to retrieve
    recommendations = find_similar_products(es_id, num_results)

    not_relevant = []
    for pro in response['hits']['hits']:
        if pro['_source']['Product Name'] != hit['_source']['Product Name']:
            found_match = False
            for others in recommendations:
                if pro['_source']['Product Name'] == others['Product Name']:
                    found_match = True
            if found_match == False:
                not_relevant.append(pro)

    for j, rec in enumerate(recommendations):
        print(f'rank: {j+1}:', rec['Product Name'], rec.get('Product Price'))

    for i, not_relev in enumerate(not_relevant):
        print(f'rank: {j+2}:',  not_relev['_source']['Product Name'])
    print("")

<ipython-input-51-0ad655c68c1f>:39: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific paremeters.
  response = es.search(index='rand_index', body={"query": {"match_all": {}}}, size=20)


product: Super Mario Odyssey - US Version
rank: 1: Super Mario 3D World + Bowser's Fury - US Version None
rank: 2: Mario Party Superstars - US Version 53
rank: 3: Kirby and the Forgotten Land - US Version 56
rank: 4: The Legend of Zelda: Breath of the Wild - US Version 45
rank: 5: The Legend of Zelda: Tears of the Kingdom - Nintendo Switch (International Version) 59
rank: 6: Logitech G Pro Wireless Gaming Mouse with Esports Grade Performance, Black
rank: 6: RoboCop Rogue City (PS5)
rank: 6: Nintendo Neon Purple/ Neon Orange Joy-Con (L-R) - Switch
rank: 6: Logitech G923 Racing Wheel and Pedals, TRUEFORCE up to 1000 Hz Force Feedback, Responsive Driving Design, Dual Clutch Launch Control, Genuine Leather Wheel Cover, for PS5, PS4, PC, Mac - Black
rank: 6: Xbox Wireless Headset – Xbox Series X|S, Xbox One, and Windows Devices
rank: 6: Logitech G920 Driving Force Racing Wheel and Floor Pedals, Real Force Feedback, Stainless Steel Paddle Shifters, Leather Steering Wheel Cover for Xbox Serie

In [52]:
import numpy as np
import json

# Function to truncate product names
def truncate_name(name, length=20):
    return (name[:length] + '..') if len(name) > length else name

# Load the JSON data
with open('rand_data.json', 'r') as file:
    data = json.load(file)

# Extracting product names
product_names = [item['Product Data']['Product Name'] for item in data]

# Creating a mapping of product names to indices
product_index_map = {name: index for index, name in enumerate(product_names)}

# Number of products
num_products = len(product_names)

# Initialize the adjacency matrix with NaNs
adjacency_matrix = np.full((num_products, num_products), np.nan)

# Query Elasticsearch for each product
response = es.search(index='rand_index', body={"query": {"match_all": {}}}, size=20)

# Populate the adjacency matrix with actual ranks
for hit in response['hits']['hits']:
    es_id = hit['_id']
    recommendations = find_similar_products(es_id, num_products)
    product_index = product_index_map[hit['_source']['Product Name']]
    dissimilar_rank = len(recommendations) + 1  # Rank for dissimilar products

    # Assign ranks to similar products
    for rank, rec in enumerate(recommendations, start=1):
        rec_product_index = product_index_map[rec['Product Name']]
        adjacency_matrix[product_index][rec_product_index] = rank

    # Assign the same rank to dissimilar products
    for j in range(num_products):
        if np.isnan(adjacency_matrix[product_index][j]):
            adjacency_matrix[product_index][j] = dissimilar_rank

    # Set the diagonal (same product) to 0
    np.fill_diagonal(adjacency_matrix, 0)

# Print the updated matrix with truncated product names
print("Adjacency Matrix (Product Rankings):\n")
print(" ", end="\t")
for name in product_names:
    print(truncate_name(name), end="\t")
print()

for i, row in enumerate(adjacency_matrix):
    print(truncate_name(product_names[i]), end="\t")
    for rank in row:
        print(rank, end="\t")
    print()


<ipython-input-52-de2422c4b089>:25: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific paremeters.
  response = es.search(index='rand_index', body={"query": {"match_all": {}}}, size=20)


Adjacency Matrix (Product Rankings):

 	Super Mario Odyssey ..	Logitech G Pro Wirel..	The Legend of Zelda:..	RoboCop Rogue City (..	Super Mario 3D World..	Kirby and the Forgot..	Nintendo Neon Purple..	Logitech G923 Racing..	Xbox Wireless Headse..	Logitech G920 Drivin..	NexiGo PS5 Slient Co..	God of War Ragnarök ..	Star Wars: Heritage ..	Mario Party Supersta..	Logitech G335 Wired ..	Sonic Superstars - N..	NBA 2K24 Kobe Bryant..	BANDAI NAMCO Enterta..	The Legend of Zelda:..	MARVEL’S SPIDER-MAN ..	
Super Mario Odyssey ..	0.0	6.0	4.0	6.0	1.0	3.0	6.0	6.0	6.0	6.0	6.0	6.0	6.0	2.0	6.0	6.0	6.0	6.0	5.0	6.0	
Logitech G Pro Wirel..	6.0	0.0	6.0	6.0	6.0	6.0	6.0	4.0	2.0	5.0	3.0	6.0	6.0	6.0	1.0	6.0	6.0	6.0	6.0	6.0	
The Legend of Zelda:..	5.0	8.0	0.0	8.0	7.0	2.0	8.0	8.0	8.0	8.0	8.0	3.0	8.0	6.0	8.0	8.0	8.0	4.0	1.0	8.0	
RoboCop Rogue City (..	4.0	4.0	4.0	0.0	4.0	4.0	4.0	3.0	4.0	4.0	2.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	1.0	
Super Mario 3D World..	1.0	6.0	4.0	6.0	0.0	3.0	6.0	6.0	6.0	6.0	6.0	6.0	6.0	2.0	6.0	

In [53]:
# Code to save the adjacency matrix in a JSON file

import json

# Convert the numpy matrix to a list of lists for JSON serialization
matrix_list = adjacency_matrix.tolist()

# Create a dictionary with product names as keys and their corresponding rows as values
matrix_dict = {truncate_name(product_names[i]): matrix_list[i] for i in range(num_products)}

output_file = 'adjacency_matrix.json'

# Write the dictionary to a JSON file
with open(output_file, 'w') as file:
    json.dump(matrix_dict, file, indent=4)

output_file

'adjacency_matrix.json'

In [54]:
matrix_dict

{'Super Mario Odyssey ..': [0.0,
  6.0,
  4.0,
  6.0,
  1.0,
  3.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  2.0,
  6.0,
  6.0,
  6.0,
  6.0,
  5.0,
  6.0],
 'Logitech G Pro Wirel..': [6.0,
  0.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  4.0,
  2.0,
  5.0,
  3.0,
  6.0,
  6.0,
  6.0,
  1.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0],
 'The Legend of Zelda:..': [7.0,
  7.0,
  1.0,
  7.0,
  7.0,
  2.0,
  5.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  4.0,
  7.0,
  6.0,
  3.0,
  7.0,
  7.0,
  0.0,
  7.0],
 'RoboCop Rogue City (..': [4.0,
  4.0,
  4.0,
  0.0,
  4.0,
  4.0,
  4.0,
  3.0,
  4.0,
  4.0,
  2.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  1.0],
 'Super Mario 3D World..': [1.0,
  6.0,
  4.0,
  6.0,
  0.0,
  3.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  2.0,
  6.0,
  6.0,
  6.0,
  6.0,
  5.0,
  6.0],
 'Kirby and the Forgot..': [3.0,
  10.0,
  1.0,
  10.0,
  5.0,
  0.0,
  10.0,
  8.0,
  6.0,
  9.0,
  7.0,
  10.0,
  10.0,
  4.0,
  10.0,
  10.0,
  10.0,
  10.0,
  2.

In [55]:
with open('adjacency_matrix_ground_truth.json', 'r') as file:
    gt_matrix_dict = json.load(file)
gt_matrix_dict

{'Super Mario Odyssey ..': [0.0,
  9.0,
  4.0,
  6.0,
  1.0,
  3.0,
  8.0,
  7.0,
  9.0,
  9.0,
  9.0,
  5.0,
  6.0,
  2.0,
  9.0,
  5.0,
  7.0,
  9.0,
  5.0,
  6.0],
 'Logitech G Pro Wirel..': [8.0,
  0.0,
  8.0,
  6.0,
  7.0,
  6.0,
  4.0,
  1.0,
  3.0,
  1.0,
  4.0,
  8.0,
  8.0,
  8.0,
  2.0,
  8.0,
  8.0,
  5.0,
  8.0,
  8.0],
 'The Legend of Zelda:..': [4.0,
  8.0,
  0.0,
  4.0,
  2.0,
  3.0,
  7.0,
  8.0,
  8.0,
  8.0,
  8.0,
  5.0,
  3.0,
  3.0,
  8.0,
  4.0,
  6.0,
  8.0,
  1.0,
  5.0],
 'RoboCop Rogue City (..': [4.0,
  5.0,
  5.0,
  0.0,
  5.0,
  2.0,
  3.0,
  6.0,
  3.0,
  7.0,
  3.0,
  2.0,
  5.0,
  6.0,
  7.0,
  7.0,
  5.0,
  7.0,
  7.0,
  1.0],
 'Super Mario 3D World..': [1.0,
  8.0,
  3.0,
  7.0,
  0.0,
  4.0,
  7.0,
  9.0,
  9.0,
  9.0,
  9.0,
  6.0,
  6.0,
  2.0,
  9.0,
  6.0,
  7.0,
  9.0,
  5.0,
  6.0],
 'Kirby and the Forgot..': [1.0,
  5.0,
  1.0,
  2.0,
  1.0,
  0.0,
  7.0,
  7.0,
  7.0,
  7.0,
  6.0,
  4.0,
  5.0,
  1.0,
  7.0,
  5.0,
  6.0,
  7.0,
  3.0,
  6.0]

In [56]:
import numpy as np

# Function to calculate Mean Average Precision (MAP)
def mean_average_precision(system_matrix, gt_matrix):
    average_precisions = []
    for system_row, gt_row in zip(system_matrix, gt_matrix):
        gt_order = np.argsort(gt_row)
        ordered_system_predictions = system_row[gt_order]
        precisions = [np.mean(ordered_system_predictions[:k+1]) for k in range(len(ordered_system_predictions))]
        average_precision = np.mean(precisions)
        average_precisions.append(average_precision)
    return np.mean(average_precisions)

# Function to calculate Root Mean Square Error (RMSE)
def root_mean_square_error(system_matrix, gt_matrix):
    return np.sqrt(np.mean((system_matrix - gt_matrix) ** 2))


In [57]:
with open('adjacency_matrix_ground_truth.json', 'r') as file:
    gt_matrix_dict = json.load(file)  # Make sure to use a different variable name

# Convert both dictionaries to NumPy arrays
system_matrix = np.array(list(matrix_dict.values()))
gt_matrix = np.array(list(gt_matrix_dict.values()))

# Check if matrices have the same shape
if system_matrix.shape != gt_matrix.shape:
    raise ValueError("Matrices have different shapes.")

In [58]:
map_score = mean_average_precision(system_matrix, gt_matrix)
rmse_score = root_mean_square_error(system_matrix, gt_matrix)

print("MAP:", map_score)
print("RMSE:", rmse_score)


MAP: 3.525543921739641
RMSE: 2.2224926736010913


**MAP (3.53)**: MAP is typically used in information retrieval and measures the precision of retrieval for each query. It averages the precision scores across all queries (or items, in your case). In the context of our adjacency matrices, a higher MAP would generally be better, indicating that the system's predictions are more aligned with the ground truth. So the score of 3.53 show the system is working fairly good.

**RMSE (2.22)**: RMSE measures the average magnitude of errors between the system's predictions and the actual values (ground truth). It's a measure of how far, on average, the predictions are from the actual values. Lower RMSE values are better, indicating that the predictions are closer to the true values. An RMSE of 2.22 suggests that, on average, the system's predictions are about 2.22 units away from the actual values. According to the nature of our system to rank the similars from 1 moving forward, RMSE of 2.22 shows the system is working great in this way.

In [24]:
def retrieve_documents(query, index_name, num_results=10):
    search_query = {
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["Product Name^2", "Product Description^4", "Product Rating", "Product Price"]
                    }
                },
            }
        },
        "size": num_results,
        "_source": ["Product Name", "Product Price", "Product Rating", "Product Description"]
    }
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

user_query = "Spider-Man game in range 50 $"
retrieved_docs = retrieve_documents(user_query, 'total_index')
for doc in retrieved_docs:
    print(doc)


{'Product Name': 'MARVEL’S SPIDER-MAN 2 – PS5 Launch Edition', 'Product Price': '69', 'Product Rating': '4.8', 'Product Description': ' Launch Edition Includes: Arachknight Suit for Peter early unlock (includes three color variants), Shadow-Spider Suit for Miles early unlock (includes three color variants), Web Grabber gadget early unlock, +3 Skill Points Spider-Men, Peter Parker and Miles Morales, return for an exciting new adventure in the critically acclaimed Marvel’s Spider-Man franchise. Swing, jump, and utilize the new Web Wings to travel across Marvel’s New York, quickly switching between Peter Parker and Miles Morales to experience different stories and epic new powers, as the iconic villain Venom threatens to destroy their lives, their city, and the ones they love. An evolution of the Spider-Man story: The incredible power of the symbiote forces Peter and Miles to face the ultimate test of strength, both inside and outside the mask, as they balance their lives, friendships, an